In [ ]:
def KSVD(S):
    # number of columns of the dictionary
    N = natom
    M = S.shape[0]
    npatch = S.shape[1]

    # intialize the dictionary
    D = np.random.random((M, N))
    # normalize each column of D (zero mean and unit norm)
    for i in range(N):
        D[:,i] = D[:,i] - D[:,i].mean()
    D = normalize(D, axis=0, norm='l2')

    # number of iteration of the KSVD
    max_iter = niter_dl

    # initialize the coefficient matrix
    X = np.zeros((N, npatch))
    for iter in range(max_iter):
        time_start = time.time()
        print(f'iter {iter}')
        # perform the sparse coding via OMP of all the columns of S
        for n in range(npatch):
            X[:, n] = OMP(S[:, n], D, L, 1e-3)
    
        # iterate over the columns of D
        for j in range(N):
            # find which signals uses the j-th atom in the sparse coding
            omega = np.where(X[j, :] != 0)[0]

            if len(omega) == 0:
                # if the atom is never used then ignore or substitute it with a random vector
                D[:, j] = np.random.randn(M)
            else:
                # compute the residual matrix E, ignoring the j-th atom
                not_j = list(set(range(N)) - {j}) 

                D_not_j = D[:, not_j]
                X_not_j = X[not_j, :]

                E = S - D_not_j @ X_not_j

                # restrict E to the columns indicated by omega
                Eomega = E[:,omega]

                # compute the SVD of Eomega
                U, Sigma, V = np.linalg.svd(Eomega, full_matrices=False)

                # update the dictionary
                D[:,j] = U[:,0]

                # update the coefficient matrix
                X[j, omega] = Sigma[0] * V[0,:]
    
        time_end = time.time()
        print(f'{time_end - time_start:.0f}')
